This notebook copies a table from a lakehouse to another based on a timestamp.
All Delta logs and parquet files, modifyed after the provided timestamp will be removed from the target table to ensure data consistency in a precise point in time.
It considers the timestamp in which the folder or the files has been created in the source.

**Parameters** </br>
workspace_source : String - Name of the source workspace</br>
workspace_target : String - Name of the target workspace</br>
lakehouse_source : String - Name of the source lakehouse</br>
lakehouse_target = String - Name of the target lakehouse</br>
table_source = String - source table to copy from</br>
table_target = String - source table to copy to</br>
datetime_str : Timestamp - watermark for the data</br>

In [ ]:
%%spark
val workspace_source = ""
val workspace_target = ""
val lakehouse_source = ""
val lakehouse_target = ""
val table_source = ""
val table_target = ""
val datetime_str = ""

In [ ]:
%%spark
//do not change this
val source = s"abfss://$workspace_source@onelake.dfs.fabric.microsoft.com/$lakehouse_source.Lakehouse/Tables/$table_source"
val destination = s"abfss://$workspace_target@onelake.dfs.fabric.microsoft.com/$lakehouse_target.Lakehouse/Tables/$table_target"

In [ ]:
%%spark
import java.sql.Timestamp
val scalaTimestamp = Timestamp.valueOf(datetime_str)
val unixTimestamp = scalaTimestamp.getTime


In [ ]:
%%spark
mssparkutils.fs.cp(source, destination, true)

val filesToDelete = mssparkutils.fs.ls(s"$source/_delta_log")
    .filter{sf => sf.isFile && sf.modifyTime > unixTimestamp}

for(fileToDelte <- filesToDelete) {
    val destFileToDelete = s"$destination/_delta_log/${fileToDelte.name}"
    println(s"Deleting file $destFileToDelete")
    mssparkutils.fs.rm(destFileToDelete, false)
}
